In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# import transformers
# import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import RandomSampler
from datasets import load_dataset
from dataclasses import dataclass

ModuleNotFoundError: No module named 'torch'

In [2]:
import sys 
sys.path.append('/home/jz17d/anaconda3/envs/simcse/lib/python3.6/site-packages')


In [3]:
import pandas as pd
import openpyxl

# read csv, remove None, save into txt

In [22]:
# https://allenai.org/data/propara
# df = pd.read_excel('./ProPara.xlsx',sheet_name='Paragraphs', engine='openpyxl')
df = pd.read_excel('./ProPara.xlsx',sheet_name='Additional Unlabeled Paragraphs', engine='openpyxl')
# df = df.fillna('')
df

,Topic,Prompt,Paragraph ID,Sentence1,Sentence2,Sentence3,Sentence4,Sentence5,Sentence6,Sentence7,Sentence8,Sentence9,Sentence10
0,battery,How does does a battery produce electricity?,2089,Chemicals react to metals inside the battery.,Excess electrons build up on the negative elec...,A shortage of electrons of the positive electr...,Excess electrons flow out of the bttery via th...,Electrons flow through the circuit.,Electrons flow back into the battery via the p...,None,None,None,None
1,battery,How does does a battery produce electricity?,2092,A battery is a chemical.,The chemical medium allows for the flow of cha...,The device is connected to both ends.,Chemical reactions occur on the electrodes.,This creates a flow of electrical current to t...,None,None,None,None,None
2,protein synthesis,How are proteins synthesized in a cell?,2093,Information from the DNA is copied to a RNA mo...,The RNA molecule is sent to cytoplasm.,This i the messenger RNA or mRNA and it goes t...,Transfer RNA or tRNA bring amino acids to the ...,Amino acids bind together to form a chain.,Ribosome sub-units separate from each other.,None,None,None,None
3,protein synthesis,How are proteins synthesized in a cell?,2095,Transcription takes the information.,DNA is encoded into mRNA.,MRNA leaves the cell nucleus.,MRNA enters the cytoplasm.,MRNA synthesizes proteins.,None,None,None,None,None
4,protein synthesis,How are proteins synthesized in a cell?,2096,The process is initiated in the cell's nucleus.,Specific enzymes unwind the needed section of ...,The DNA in this region becomes accessible.,An RNA copy is made.,An RNA molecule then moves from the nucleus to...,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,igneous rock,How does igneous rock form?,3175,Metamorphic rocks melt.,Turn into magma.,A volcano erupts.,Magma flows out of the volcano.,The magma becomes lava when it reaches the ear...,The lava cools and hardens.,Becomes igneous rock.,None,None,None
873,igneous rock,How does igneous rock form?,3176,A volcano erupts.,Superheated magma flows up through the volcano...,The magma flows outside of the volcano hitting...,This is lava.,The lava cools.,It forms igneous rock.,None,None,None,None
874,pancreas,Describe how the pancreas works,3178,The pancreas produces trypsin and chymotrypsin...,The pancreas produces amylase to break down ca...,The pancreas produces lipase to break down fats.,The pancreas produces insulin.,Insulin moves glucose from the blood to muscle...,The pancreas produces glucagon.,Glucagon breaks down glycogen in the liver.,None,None,None
875,pancreas,Describe how the pancreas works,3179,Enzymes come from the stomach.,The pancreas uses digestive juices.,The digestive juices dissolve food.,The smaller pieces of food are dissolved.,The pancreas also regulates insulin.,None,None,None,None,None


In [23]:
cols = []
for i in range(1,11):
    col = 'Sentence'+str(i)
    cols.append(col)

In [37]:
paras_with_None = df[cols].values.tolist()

In [38]:
paras = []
for sublist in paras_with_None:
    cleaned = [elem for elem in sublist if elem is not None]
    if len(cleaned):  # anything left?
        paras.append(cleaned)

In [45]:
with open('paras.txt','w') as f:
    for para in paras:
        f.writelines(str(para)+'\n')

In [46]:
f = open('paras.txt','r')
f.readlines()

["['Chemicals react to metals inside the battery.', 'Excess electrons build up on the negative electrode.', 'A shortage of electrons of the positive electrode.', 'Excess electrons flow out of the bttery via the negative terminal.', 'Electrons flow through the circuit.', 'Electrons flow back into the battery via the positive terminal.']\n",
 "['A battery is a chemical.', 'The chemical medium allows for the flow of charge between the anode and cathode.', 'The device is connected to both ends.', 'Chemical reactions occur on the electrodes.', 'This creates a flow of electrical current to the device.']\n",
 "['Information from the DNA is copied to a RNA molecule.', 'The RNA molecule is sent to cytoplasm.', 'This i the messenger RNA or mRNA and it goes to a ribosome.', 'Transfer RNA or tRNA bring amino acids to the ribosome.', 'Amino acids bind together to form a chain.', 'Ribosome sub-units separate from each other.']\n",
 "['Transcription takes the information.', 'DNA is encoded into mRNA.

# load with huggingface load_dataset

In [4]:
import collections

In [5]:
datasets = load_dataset('text', data_files='paras.txt', cache_dir="./data/")


Using custom data configuration default
Reusing dataset text (./data/text/default-45dca5bb88cef864/0.0.0/daf90a707a433ac193b369c8cc1772139bb6cca21a9c7fe83bdd16aad9b9b6ab)


# preprocessing training data

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [7]:
def eval_str_2_list(examples):
    paragraphs = examples['text']
    res = {'text':[]}
    for paragraph in paragraphs:
        res['text'].append(eval(paragraph))
    return res

In [8]:
def f(examples): # only works for unsupervised
    # examples is a dict of paragraphs of sentences
    paragraphs = examples['text']
    num_para = len(paragraphs)
    total = 0
    para_features = collections.defaultdict(list)
    para_lengths = []
    for paragraph in paragraphs:
        para_lengths.append(len(paragraph))
        sent_features = tokenizer(
            paragraph,
            max_length=32,
            truncation=True,
            padding=False
        )
        for key in sent_features:
            para_features[key].append([[sent_features[key][i] ,sent_features[key][i]] for i in range(len(sent_features[key]))])

    return para_features

In [9]:
train_dataset = datasets["train"].map(eval_str_2_list, batched = True).map(f, batched = True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
sum([7, 5, 6, 5, 5, 7, 9, 6, 8, 5])

63

In [691]:
class ParagraphLoader(DataLoader):

    def __init__(self, tokenizer, dataset, batchsize_threshold = 64, drop_last=False):
        self.ds = dataset
        self.batchsize_threshold = batchsize_threshold
        self.drop_last = drop_last
        self.sampler = RandomSampler(self.ds)
        self.length = len(dataset)
        self.used = 0
        self.tokenizer = tokenizer
            
    def __iter__(self):
        batch = collections.defaultdict(list)
        batch_para_length = []
        prev_size = 0
        
        # random select an idx(paragraph)
        for idx in self.sampler:
            self.used += 1
            new_para = self.ds[idx] # a paragraph is a list of sentence pairs
            new_para_len = len(new_para['input_ids']) 
            num_sent = len(new_para['input_ids'][0]) # NOT total number of sentences, but num of sentences in each pa
            if prev_size + new_para_len <= self.batchsize_threshold:
                prev_size += new_para_len
                for key in new_para:
                    if key == 'text':
                        continue
                    for i in range(new_para_len):
                        batch[key] += new_para[key][i]
                
                batch_para_length.append(new_para_len)
            else:
                batch_out = tokenizer.pad(
                    batch,
                    padding=True, # 
                    max_length=32, # 
                    return_tensors="pt",
                )
                
#                 if model_args.do_mlm:
                if True:
                    batch_out["mlm_input_ids"], batch_out["mlm_labels"] = mask_tokens(tokenizer, batch_out["input_ids"])
                batch_out = {k: batch_out[k].view(batch_out["mlm_input_ids"].shape[0]//2, 2, -1) for k in batch_out}
#                 batch_out['batch_para_length'] = torch.Tensor(batch_para_length)
                batch_out['batch_para_length'] = batch_para_length
                
                yield self.used, batch_out
                
                batch = collections.defaultdict(list)
                for key in new_para:
                    if key == 'text':
                        continue
                    for i in range(new_para_len):
                        batch[key] += new_para[key][i]
                prev_size = new_para_len
                batch_para_length = [new_para_len]
                
        # the last batch
        if len(batch)>0 and not self.drop_last:
            batch_out = tokenizer.pad(
                    batch,
                    padding=True, # 
                    max_length=32, # 
                    return_tensors="pt",
                )
#             if model_args.do_mlm:
            if True:
                batch_out["mlm_input_ids"], batch_out["mlm_labels"] = mask_tokens(tokenizer, batch_out["input_ids"])
            batch_out = {k: batch_out[k].view(batch_out["mlm_input_ids"].shape[0]//2, 2, -1) for k in batch_out}
            batch_out['batch_para_length'] = batch_para_length
#             batch_out['batch_para_length'] = torch.Tensor(batch_para_length)
            
            yield self.used, batch_out 
            
    def __len__(self):
        return self.length

In [692]:
train_dataloader = ParagraphLoader(tokenizer, train_dataset, 64)

In [693]:
for i,j in train_dataloader:
    print(i)
#     break

12
22
32
42
52
62
73
84
94
104
114
125
135
144
154
164
173
183
194
203
211
220
229
240
251
260
271
281
293
302
311
320
330
340
350
360
369
379
388
398
407
416
425
434
444
454
463
472
482
492
502
513
524
535
544
555
565
573
583
593
604
614
624
635
644
654
664
674
684
694
705
715
725
734
743
752
762
774
784
794
805
816
825
835
845
855
864
874
877


In [542]:
def mask_tokens(tokenizer, inputs: torch.Tensor, special_tokens_mask: Optional[torch.Tensor] = None
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original.
    """
    inputs = inputs.clone()
    labels = inputs.clone()
    mlm_probability = 0.8
    # We sample a few tokens in each sequence for MLM training (with probability `mlm_probability`)
    probability_matrix = torch.full(labels.shape, mlm_probability)
    if special_tokens_mask is None:
        special_tokens_mask = [
            tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
        ]
        special_tokens_mask = torch.tensor(special_tokens_mask, dtype=torch.bool)
    else:
        special_tokens_mask = special_tokens_mask.bool()

    probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # We only compute loss on masked tokens

    # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
    indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
    inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    # 10% of the time, we replace masked input tokens with random word
    indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
    random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
    inputs[indices_random] = random_words[indices_random]

    # The rest of the time (10% of the time) we keep the masked input tokens unchanged
    return inputs, labels

In [697]:
batch_para_length = j['batch_para_length']
cumlen = 0
labels = torch.zeros((sum(batch_para_length), sum(batch_para_length)))
for para_len in batch_para_length:
    para_label = torch.diag(0.5*torch.ones(para_len-1), -1) + torch.diag(0.5*torch.ones(para_len-1), 1)+torch.diag(torch.ones(para_len))
    labels[cumlen:cumlen+para_len,cumlen:cumlen+para_len] = para_label
    cumlen += para_len

In [685]:
j['batch_para_length'].items()

AttributeError: 'Tensor' object has no attribute 'items'

# data collator

In [334]:
@dataclass
class OurDataCollatorWithPadding:

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    mlm: bool = True
    mlm_probability: float = 0.2

    def __call__(self, features: List[Dict[str, Union[List[int], List[List[int]], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # features dim: (bs, para, sent, word), need to 
        special_keys = ['input_ids', 'attention_mask', 'token_type_ids', 'mlm_input_ids', 'mlm_labels']
        bs = len(features)
        if bs > 0:
            num_para = len(features[0]['input_ids'])
        else:
            return
        flat_features = []
        batch_para_length = []
        for feature in features:
            for i in range(num_sent):
                flat_feature = {}
                for k in feature:
                    if k == 'batch_para_length':
                        if i == 0:
                            batch_para_length.append([eval(feature[k])])
                    elif k in special_keys:
                        flat_feature[k] = feature[k][i]
                    else:
                        flat_feature[k] = feature[k]
                flat_features.append(flat_feature)
        batch = self.tokenizer.pad(
            flat_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        if model_args.do_mlm:
            batch["mlm_input_ids"], batch["mlm_labels"] = self.mask_tokens(batch["input_ids"])

        batch = {k: batch[k].view(bs, num_sent, -1) if k in special_keys else batch[k].view(bs, num_sent, -1)[:, 0] for k in batch}
        batch["batch_para_length"] = torch.Tensor(batch_para_length)
        if "label" in batch:
            batch["labels"] = batch["label"]
            del batch["label"]
        if "label_ids" in batch:
            batch["labels"] = batch["label_ids"]
            del batch["label_ids"]

        return batch

    def mask_tokens(
        self, inputs: torch.Tensor, special_tokens_mask: Optional[torch.Tensor] = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original.
        """
        inputs = inputs.clone()
        labels = inputs.clone()
        # We sample a few tokens in each sequence for MLM training (with probability `self.mlm_probability`)
        probability_matrix = torch.full(labels.shape, self.mlm_probability)
        if special_tokens_mask is None:
            special_tokens_mask = [
                self.tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
            ]
            special_tokens_mask = torch.tensor(special_tokens_mask, dtype=torch.bool)
        else:
            special_tokens_mask = special_tokens_mask.bool()

        probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
        masked_indices = torch.bernoulli(probability_matrix).bool()
        labels[~masked_indices] = -100  # We only compute loss on masked tokens

        # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
        indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
        inputs[indices_replaced] = self.tokenizer.convert_tokens_to_ids(self.tokenizer.mask_token)

        # 10% of the time, we replace masked input tokens with random word
        indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
        random_words = torch.randint(len(self.tokenizer), labels.shape, dtype=torch.long)
        inputs[indices_random] = random_words[indices_random]

        # The rest of the time (10% of the time) we keep the masked input tokens unchanged
        return inputs, labels

In [332]:
from transformers.tokenization_utils_base import BatchEncoding, PaddingStrategy, PreTrainedTokenizerBase
from typing import Optional, Union, List, Dict, Tuple

In [523]:
import numpy as np

In [531]:
flatten_features = []

for i in range(3):
    flat = {}
    r = np.random.randint(1,10)
    for k in ['input_ids', 'attention_mask', 'token_type_ids', 'mlm_input_ids', 'mlm_labels']:
        flat[k] = [1]*r
    flatten_features.append(flat)

In [532]:
flatten_features

[{'input_ids': [1, 1, 1],
  'attention_mask': [1, 1, 1],
  'token_type_ids': [1, 1, 1],
  'mlm_input_ids': [1, 1, 1],
  'mlm_labels': [1, 1, 1]},
 {'input_ids': [1, 1, 1, 1, 1],
  'attention_mask': [1, 1, 1, 1, 1],
  'token_type_ids': [1, 1, 1, 1, 1],
  'mlm_input_ids': [1, 1, 1, 1, 1],
  'mlm_labels': [1, 1, 1, 1, 1]},
 {'input_ids': [1, 1, 1, 1, 1, 1, 1],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1],
  'token_type_ids': [1, 1, 1, 1, 1, 1, 1],
  'mlm_input_ids': [1, 1, 1, 1, 1, 1, 1],
  'mlm_labels': [1, 1, 1, 1, 1, 1, 1]}]

In [535]:
tokenizer.pad(
            flatten_features[0],
            padding=True,
            max_length=32,
            return_tensors="pt",
        )

{'input_ids': tensor([1, 1, 1]), 'attention_mask': tensor([1, 1, 1]), 'token_type_ids': tensor([1, 1, 1]), 'mlm_input_ids': tensor([1, 1, 1]), 'mlm_labels': tensor([1, 1, 1])}

In [316]:
for i,j in train_dataloader:
    print('#'*10)
    print(i)

##########
21
##########
31
##########
39
##########
49
##########
59
##########
68
##########
78
##########
88
##########
98
##########
108
##########
119
##########
129
##########
138
##########
148
##########
157
##########
168
##########
178
##########
189
##########
199
##########
210
##########
218
##########
227
##########
237
##########
247
##########
258
##########
268
##########
278
##########
287
##########
298
##########
307
##########
316
##########
327
##########
336
##########
346
##########
356
##########
366
##########
375
##########
384
##########
394
##########
403
##########
414
##########
424
##########
434
##########
444
##########
454
##########
465
##########
474
##########
485
##########
495
##########
504
##########
515
##########
525
##########
535
##########
546
##########
556
##########
565
##########
575
##########
585
##########
595
##########
605
##########
616
##########
627
##########
637
##########
647
##########
658
##########
668
##########
678
####

In [317]:
j['attention_mask']

[[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]],
  [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]],
 [[[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
   [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
   [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]],
  [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [307]:
torch.Tensor(j['attention_mask']).size(0)

ValueError: expected sequence of length 13 at dim 3 (got 9)

In [304]:
j

defaultdict(list,
            {'attention_mask': [[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
               [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]],
              [[[1, 1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 1, 1, 1],
                [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [240]:
labels = torch.zeros((sum(batch_para_length), sum(batch_para_length)))
cumlen = 0
for para_len in batch_para_length:
    para_label = torch.diag(0.5*torch.ones(para_len-1), -1) + torch.diag(0.5*torch.ones(para_len-1), 1)+torch.diag(torch.ones(para_len))
    labels[cumlen:cumlen+para_len,cumlen:cumlen+para_len] = para_label
    cumlen += para_len

In [241]:
labels

tensor([[1.0000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 1.0000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.0000, 0.5000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.5000, 1.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.5000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 1.0000, 0.5000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 1.0000]])

In [ ]:
labels = torch.arange(cos_sim.size(0)).long().to(cls.device)

# pos

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# import torch.distributed as dist

import transformers
from transformers import RobertaTokenizer
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel, RobertaModel, RobertaLMHead
from transformers.models.bert.modeling_bert import BertPreTrainedModel, BertModel, BertLMPredictionHead
from transformers.activations import gelu
from transformers.file_utils import (
    add_code_sample_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    replace_return_docstrings,
)
from transformers.modeling_outputs import SequenceClassifierOutput, BaseModelOutputWithPoolingAndCrossAttentions


In [1]:
from transformers import RobertaTokenizer

SimCSE use consine similarity. Euclidean distance may also work.

In [343]:
column_names = datasets["train"].column_names
pos_labels = datasets["train"]['pos_labels']

sent2_cname = None
if len(column_names) == 3:
    # Pair datasets
    sent0_cname = column_names[1]
    sent1_cname = column_names[1]
elif len(column_names) == 4:
    # Pair datasets with hard negatives
    sent0_cname = column_names[0]
    sent1_cname = column_names[1]
    sent2_cname = column_names[2]
elif len(column_names) == 2:
    # Unsupervised datasets
    sent0_cname = column_names[0]
    sent1_cname = column_names[0]

In [344]:
for k in feature:
    if k in special_keys:
        k: feature[k][i]
    else:
        k: feature[k]
{k: feature[k][i]  else  }

'text'

In [345]:
train_dataset = datasets["train"].map(
    prepare_features,
    batched=True,
    num_proc=4,
    remove_columns=column_names.remove('pos_labels'),
    load_from_cache_file=False,
)

KeyboardInterrupt: 

In [296]:
tokens['input_ids']

tensor([[  101,  2002,  2036,  2988,  1037,  2047,  2394,  5449,  2000,  3711,
          1999, 10476,  1010,  2007,  1037,  2367,  4955,  1998, 10439, 10497,
         23522,  1012,   102]])

In [294]:
len(tokenizer)

30522

In [285]:
np.savetxt('./pos_vectors.txt', pos_vectors)

In [288]:
df = pd.DataFrame()
df['text'] = list_ds
df['pos'] = pos_vectors.tolist()
# df['']

In [297]:
for i in range(len(list_ds)):
    list_ds[i] = list_ds[i]['text']
df['text'] = list_ds


In [300]:
df['pos_labels'] = df['pos']
del df['pos']

In [301]:
df.head()

,text,pos_labels
0,YMCA in South Australia,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,South Australia (SA) has a unique position in...,"[6.0, 6.0, 0.0, 2.0, 3.0, 4.0, 0.0, 6.0, 0.0, ..."
2,"The compound of philosophical radicalism, evan...","[8.0, 4.0, 0.0, 1.0, 1.0, 3.0, 0.0, 10.0, 0.0,..."
3,It was into this social setting that in Februa...,"[1.0, 4.0, 3.0, 1.0, 0.0, 5.0, 0.0, 7.0, 1.0, ..."
4,"for apprentices and others, after their day's ...","[1.0, 2.0, 0.0, 0.0, 2.0, 1.0, 0.0, 13.0, 0.0,..."


In [337]:
df.to_csv('wiki1m_with_pos.csv', index=False)

In [303]:
datasets = load_dataset('csv', data_files='wiki1m_with_pos.csv')

Using custom data configuration default-d4b3ddfd15a0f641


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/jz17d/.cache/huggingface/datasets/csv/default-d4b3ddfd15a0f641/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [308]:
datasets['train'][0]

{'Unnamed: 0': 0,
 'text': 'YMCA in South Australia',
 'pos_labels': '[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0]'}

# multitask learning

In [93]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(5)] + [nn.Linear(10, 1) for i in range(5)])

    def forward(self, x, itask):
        # ModuleList can act as an iterable, or be indexed using ints
        x = self.linears[itask](x)
        return x

In [94]:
m = MyModule()

In [95]:
optim = AdamW(m.parameters())

In [96]:
losses = {}
for i in range(5):
    losses[i] = nn.CrossEntropyLoss()
for i in range(5,10):
    losses[i] = nn.MSELoss()


In [104]:
data = [[torch.rand(3,10), torch.LongTensor([2,1,1]), 1],[torch.rand(3,10), torch.tensor([[3.66], [2.2], [5.3]]), 8]]

In [105]:
loss = 0
for x, label, itask in data:
    print(1)
    y = m(x, itask)
    loss += losses[itask](y, label)


1
1


In [106]:
list(m.parameters())

[Parameter containing:
 tensor([[-0.3067,  0.0341,  0.0203,  0.1702, -0.2640,  0.2146,  0.0281, -0.0577,
           0.0566,  0.0228],
         [ 0.2475, -0.2335,  0.2277,  0.2726, -0.2607, -0.2502,  0.0071, -0.1531,
           0.2502, -0.2183],
         [ 0.0174,  0.2827,  0.2271, -0.2539, -0.1468,  0.1714, -0.0718, -0.2340,
           0.1172,  0.3125],
         [ 0.0779,  0.2615,  0.1721, -0.0428,  0.1457, -0.1388, -0.2618,  0.2752,
           0.2857,  0.1946],
         [ 0.0675,  0.1020, -0.1554, -0.0436,  0.0427, -0.1463,  0.0912, -0.2415,
           0.1598, -0.2396],
         [ 0.1698, -0.2069, -0.0858, -0.2780,  0.0846, -0.0972, -0.0066, -0.1409,
          -0.1431, -0.0143],
         [-0.2247, -0.3119,  0.2872, -0.2774, -0.0910, -0.0781, -0.1879,  0.0289,
           0.0398,  0.1158],
         [ 0.0516,  0.0751, -0.1403, -0.1878, -0.2761, -0.0752, -0.1612,  0.1128,
           0.1510,  0.1801],
         [ 0.1792,  0.1318, -0.0726, -0.1448, -0.1286,  0.0148,  0.1043,  0.2748,
       

In [107]:
loss.backward()

In [83]:
from transformers.optimization import Adafactor, AdamW, get_scheduler

In [108]:
optim.step()

In [109]:
optim.zero_grad()

In [110]:
list(m.parameters())

[Parameter containing:
 tensor([[-0.3074,  0.0334,  0.0196,  0.1695, -0.2647,  0.2139,  0.0274, -0.0583,
           0.0559,  0.0221],
         [ 0.2481, -0.2329,  0.2283,  0.2733, -0.2601, -0.2495,  0.0078, -0.1524,
           0.2509, -0.2176],
         [ 0.0181,  0.2834,  0.2278, -0.2532, -0.1462,  0.1721, -0.0711, -0.2333,
           0.1165,  0.3132],
         [ 0.0772,  0.2609,  0.1714, -0.0435,  0.1450, -0.1394, -0.2625,  0.2745,
           0.2851,  0.1939],
         [ 0.0669,  0.1014, -0.1560, -0.0443,  0.0420, -0.1469,  0.0905, -0.2422,
           0.1591, -0.2403],
         [ 0.1692, -0.2076, -0.0865, -0.2786,  0.0839, -0.0979, -0.0073, -0.1416,
          -0.1438, -0.0149],
         [-0.2254, -0.3126,  0.2866, -0.2781, -0.0917, -0.0788, -0.1886,  0.0282,
           0.0391,  0.1151],
         [ 0.0509,  0.0745, -0.1410, -0.1884, -0.2768, -0.0758, -0.1619,  0.1121,
           0.1503,  0.1794],
         [ 0.1786,  0.1312, -0.0732, -0.1454, -0.1293,  0.0142,  0.1036,  0.2741,
       